# Estimation d'intégrale par méthode de Monte-Carlo

## Méthode naïve (tirage par loi uniforme)

In [ ]:
from typing import Callable, Any

import numpy as np
import matplotlib.pyplot as plt

# from scipy.stats import norm as normal
# from scipy.stats import uniform

rd = np.random.default_rng(10)

In [ ]:
def compute_integ_estimate(
    fun: Callable[[Any], float], law: Callable[[int], np.ndarray], big_n: int
) -> float:
    return sum(fun(x) for x in law(big_n)) / big_n


def cos_integ_estimate_unif(big_n: int):
    return compute_integ_estimate(
        fun=lambda x: np.cos(np.pi * x / 2),
        law=lambda n: rd.uniform(low=0, high=1, size=n),
        big_n=big_n,
    )


def show_stats(mu_ech: np.ndarray | list) -> None:
    mean = sum(mu_ech) / len(mu_ech)
    var = sum((e - mean) ** 2 for e in mu_ech) / (len(mu_ech) - 1)
    print(2 / np.pi, mean, var)
    plt.hist(mu_ech, bins=30)

### 5000 tirages de $\hat{\mu}_{1000}$

In [ ]:
mu_ech = [cos_integ_estimate_unif(1000) for _ in range(5000)]
show_stats(mu_ech)

### 5000 tirages de $\hat{\mu}_1$

In [ ]:
mu_ech2 = [cos_integ_estimate_unif(1) for _ in range(5000)]
show_stats(mu_ech2)

## Modification de la loi de tirage

In [ ]:
def new_law(
    base_law: Callable[[], float],
    majorant: float,
    fun: Callable[[float], float],
    n_ech: int,
) -> np.ndarray:
    res = np.empty((n_ech,), dtype=float)
    current_index = 0
    while current_index < n_ech:
        while fun(candidate := base_law()) < majorant * rd.uniform(low=0, high=1):
            pass
        res[current_index] = candidate
        current_index += 1
    return res


def q_tilde(number: float | np.ndarray) -> float | np.ndarray:
    return 3 * (1 - number**2) / 2


def modified_cos(number: float | np.ndarray) -> float | np.ndarray:
    return np.cos(np.pi * number / 2) / q_tilde(number)


def modified_law(n_ech: int) -> np.ndarray:
    return new_law(
        base_law=lambda: rd.uniform(low=0, high=1),
        majorant=3 / 2,
        fun=q_tilde,
        n_ech=n_ech,
    )


def cos_integ_estimate_reject(big_n: int):
    return compute_integ_estimate(
        fun=modified_cos,
        law=modified_law,
        big_n=big_n,
    )

### 2000 tirages de $\hat{\mu}^2_{1000}$

In [ ]:
reject_mu_ech = [cos_integ_estimate_reject(1000) for _ in range(2000)]
show_stats(reject_mu_ech)

#### Pour finir le TP

Pour les 2 méthodes :

- Calculer intégrale
- On calule plein de $\hat{\mu}_n$, on checke avec la limite théorique
- On checke que ça marche avec le TCL
- Faire un petit bilan (qu'est-ce qu'on va utiliser au final, pourquoi, tout ça)

Probabilité d'acceptation théorique, empirique
